In [ ]:
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime as dt
import numpy as np

In [33]:
### all the determined variables
jrds_dir = "../resultsJRDS/rds-results.csv"
result_dir = "../graphs/randdot/randdot.pdf"
control = (('ah', 'k'), ('dd', 'k'), ('ez', 'k'), ('it', 'k'), ('ll', 'k'), ('sh', 'k'), ('sm', 'k'), ('sr', 'k'))
exper = (('bb','b'), ('by','g'), ('gn','r'), ('gp','c'), ('jz', 'b'), ('mb', 'g'), ('ni','m'), ('et','y'))

In [34]:
def plot_obs(obs_set, facecolor):
    result = []
    df = pd.read_csv(jrds_dir)
    for obs, color in obs_set:
        for size, marker in [('Small', 'o'), ('Middle', 'v'), ('Big', 's')]:

            curr_df = df[(df['Initials'] == obs) & (df["Condition"] == size)]
            dates = {dt.strptime(x, '%d-%b-%Y'):x for x in curr_df['Date']}
            
            first_date = dates[min(dates)]
            last_date = dates[max(dates)]
            
            first = curr_df[curr_df['Date'] == first_date]['Threshold'].iloc[0]
            last = curr_df[curr_df['Date'] == last_date]['Threshold'].iloc[0]
#             print(obs)
#             print(size)
#             print(first_date)
#             print(last_date)
#             print(first)
#             print(last)
            if facecolor:
                plt.plot(first, last, color=color, marker=marker, label=size, linestyle='None', markerfacecolor=color,
                markeredgecolor=color, markersize=10, alpha=0.5)
            else:
                plt.plot(first, last, color=color, marker=marker, label=size, linestyle='None', markerfacecolor='none',
                markeredgecolor=color, markersize=10, alpha=0.5)

In [35]:
def plot_randdot():
    data_dir = '../resultsJRDS/randdot.csv'
    df = pd.read_csv(data_dir)
    cols = df.columns.tolist()[1:]
    for obs, color in exper:
        obs = obs.upper()
        curr_df = df[df['Observer'] == obs]
        first = get_first_num(cols, curr_df)
        last = get_first_num(reversed(cols), curr_df)
        if first != None and last != None:
            plt.plot(first, last, color=color, marker='D', label='Rand dot', linestyle='None', markersize=10, alpha=0.5)
                
#             if (np.issubdtype(curr_df[c].dtype, np.number):
#                 print(curr_df[c].item() > 0)
#             print(np.issubdtype(curr_df[c].dtype, np.number) and not curr_df[c].isnull())

In [36]:
def get_first_num(cols, curr_df):
    for c in cols:
        if pd.isnull(curr_df[c].item()):
            continue
        if not curr_df[c].dtype == np.float64 or curr_df[c].dtype == np.int64:
            continue
        return curr_df[c].iloc[0]

In [37]:
fig = plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel('Post Test', fontsize=25)
ax.set_xlabel('Pre Test', fontsize=25)

plot_randdot()
plot_obs(exper, True)
plot_obs(control, False)

# one to one line
x = np.linspace(*ax.get_xlim())
plt.plot(x, x)

ax.set_xticks([20, 200, 2000])
ax.set_yticks([20, 200, 2000])
ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

# Creating the legend for the figure
handles, labels = ax.get_legend_handles_labels()
ax.legend([handles[0]] + handles[len(exper) - 1:][:3], [labels[0]] + labels[len(exper) - 1:], loc='upper left', fontsize=18)
        
fig.savefig(result_dir, bbox_inches='tight', format='pdf', dpi=1000)
plt.close()